# Development of Intelligent Computing Systems _ 2022  IME-USP
- course [page][4]
- ministred by: MSc [Renato Cordeiro Ferreira][1]
- student: [Pedro Almeida][3] and [Rodrigo Didier Anderson][2]

[1]: https://www.linkedin.com/in/renatocf/
[2]: https://www.linkedin.com/in/didier11/
[3]: https://www.linkedin.com/in/plbalmeida/
[4]: https://www.ime.usp.br/verao/index.php

This is the first part of the course project, we will create the training pipeline for a categorization model.

More specifically, the goal is to train a model that should receive data related to products and return the best categories for them.


- More details about this stage of the project [here][1].
- More info about the data can be found [here][2]
[1]: https://github.com/didier-rda/intelligent-systems-project/blob/main/training/README.md
[2]: https://github.com/didier-rda/intelligent-systems-project/blob/main/data/README.md

## Training Pipeline  

This training pipeline follows the following steps. 

For each step, a class was created with the necessary methods to fulfill the respective stage of the pipline.

1) **Data extraction** <br>
   class: `DataExtractor`
   
   Loads a dataset with product data from a specified path available in the
   environment variable `DATASET_PATH`.
   
   
   
2) **Data formatting** <br>
   class: `DataFormatter`
   
   Processes "query" feature through CountVectorizer class from scikit-learn, training (70%) and test (30%) sets are generated in sequence.
      


3) **Modeling** <br>
   class: `Modeler`
   
   Naive Bayes classifier was the chosen model, it's commonly used for testing NLP classification problems. Therefore, the MultinomialNB class from scikit-learn was used, which works well with integer type features generated through CountVectorizer. MultinomialNB is also used for multiple label classification, which is the problem of the present dataset.
   


4) **Model validation** <br>
   class: `ModelValidator`
   
   Generates metrics about the model accuracy (precision, recall, F1, etc.)
   for each category and exports them to a specified path available in the
   environment variable `METRICS_PATH`.
   
   
   
5) **Model exportation** <br>
   class: `ModelExporter`
   
   Exports a candidate model to a specified path available in the environment variable `MODEL_PATH`.

# Import libs

In [1]:
import os
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# 1. Data Extraction

In [2]:
class DataExtractor:
    
    def __init__(self):
        self.path = os.getenv('DATASET_PATH')
        self.data = self.get_data()
    
    def get_data(self):
        return pd.read_csv(self.path)

#  2. Data Formatting

In [3]:
# source: https://gist.github.com/alopes/5358189
stop_words = [ 'a', 'à', 'adeus', 'agora', 'aí', 'ainda', 'além', 'algo', 'alguém', 'algum', 'alguma', 'algumas', 'alguns', 'ali', 'ampla', 'amplas', 'amplo', 'amplos', 'ano', 'anos', 'ante', 'antes', 'ao', 'aos', 'apenas', 'apoio', 'após', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aqui', 'aquilo', 'área', 'as', 'às', 'assim', 'até', 'atrás', 'através', 'baixo', 'bastante', 'bem', 'boa', 'boas', 'bom', 'bons', 'breve', 'cá', 'cada', 'catorze', 'cedo', 'cento', 'certamente', 'certeza', 'cima', 'cinco', 'coisa', 'coisas', 'com', 'como', 'conselho', 'contra', 'contudo', 'custa', 'da', 'dá', 'dão', 'daquela', 'daquelas', 'daquele', 'daqueles', 'dar', 'das', 'de', 'debaixo', 'dela', 'delas', 'dele', 'deles', 'demais', 'dentro', 'depois', 'desde', 'dessa', 'dessas', 'desse', 'desses', 'desta', 'destas', 'deste', 'destes', 'deve', 'devem', 'devendo', 'dever', 'deverá', 'deverão', 'deveria', 'deveriam', 'devia', 'deviam', 'dez', 'dezanove', 'dezasseis', 'dezassete', 'dezoito', 'dia', 'diante', 'disse', 'disso', 'disto', 'dito', 'diz', 'dizem', 'dizer', 'do', 'dois', 'dos', 'doze', 'duas', 'dúvida', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'embora', 'enquanto', 'entre', 'era', 'eram', 'éramos', 'és', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estás', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estiveste', 'estivestes', 'estou', 'etc', 'eu', 'exemplo', 'faço', 'falta', 'favor', 'faz', 'fazeis', 'fazem', 'fazemos', 'fazendo', 'fazer', 'fazes', 'feita', 'feitas', 'feito', 'feitos', 'fez', 'fim', 'final', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'forma', 'formos', 'fosse', 'fossem', 'fôssemos', 'foste', 'fostes', 'fui', 'geral', 'grande', 'grandes', 'grupo', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'havia', 'hei', 'hoje', 'hora', 'horas', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvéssemos', 'isso', 'isto', 'já', 'la', 'lá', 'lado', 'lhe', 'lhes', 'lo', 'local', 'logo', 'longe', 'lugar', 'maior', 'maioria', 'mais', 'mal', 'mas', 'máximo', 'me', 'meio', 'menor', 'menos', 'mês', 'meses', 'mesma', 'mesmas', 'mesmo', 'mesmos', 'meu', 'meus', 'mil', 'minha', 'minhas', 'momento', 'muita', 'muitas', 'muito', 'muitos', 'na', 'nada', 'não', 'naquela', 'naquelas', 'naquele', 'naqueles', 'nas', 'nem', 'nenhum', 'nenhuma', 'nessa', 'nessas', 'nesse', 'nesses', 'nesta', 'nestas', 'neste', 'nestes', 'ninguém', 'nível', 'no', 'noite', 'nome', 'nos', 'nós', 'nossa', 'nossas', 'nosso', 'nossos', 'nova', 'novas', 'nove', 'novo', 'novos', 'num', 'numa', 'número', 'nunca', 'o', 'obra', 'obrigada', 'obrigado', 'oitava', 'oitavo', 'oito', 'onde', 'ontem', 'onze', 'os', 'ou', 'outra', 'outras', 'outro', 'outros', 'para', 'parece', 'parte', 'partir', 'paucas', 'pela', 'pelas', 'pelo', 'pelos', 'pequena', 'pequenas', 'pequeno', 'pequenos', 'per', 'perante', 'perto', 'pode', 'pude', 'pôde', 'podem', 'podendo', 'poder', 'poderia', 'poderiam', 'podia', 'podiam', 'põe', 'põem', 'pois', 'ponto', 'pontos', 'por', 'porém', 'porque', 'porquê', 'posição', 'possível', 'possivelmente', 'posso', 'pouca', 'poucas', 'pouco', 'poucos', 'primeira', 'primeiras', 'primeiro', 'primeiros', 'própria', 'próprias', 'próprio', 'próprios', 'próxima', 'próximas', 'próximo', 'próximos', 'pude', 'puderam', 'quais', 'quáis', 'qual', 'quando', 'quanto', 'quantos', 'quarta', 'quarto', 'quatro', 'que', 'quê', 'quem', 'quer', 'quereis', 'querem', 'queremas', 'queres', 'quero', 'questão', 'quinta', 'quinto', 'quinze', 'relação', 'sabe', 'sabem', 'são', 'se', 'segunda', 'segundo', 'sei', 'seis', 'seja', 'sejam', 'sejamos', 'sem', 'sempre', 'sendo', 'ser', 'será', 'serão', 'serei', 'seremos', 'seria', 'seriam', 'seríamos', 'sete', 'sétima', 'sétimo', 'seu', 'seus', 'sexta', 'sexto', 'si', 'sido', 'sim', 'sistema', 'só', 'sob', 'sobre', 'sois', 'somos', 'sou', 'sua', 'suas', 'tal', 'talvez', 'também', 'tampouco', 'tanta', 'tantas', 'tanto', 'tão', 'tarde', 'te', 'tem', 'tém', 'têm', 'temos', 'tendes', 'tendo', 'tenha', 'tenham', 'tenhamos', 'tenho', 'tens', 'ter', 'terá', 'terão', 'terceira', 'terceiro', 'terei', 'teremos', 'teria', 'teriam', 'teríamos', 'teu', 'teus', 'teve', 'ti', 'tido', 'tinha', 'tinham', 'tínhamos', 'tive', 'tivemos', 'tiver', 'tivera', 'tiveram', 'tivéramos', 'tiverem', 'tivermos', 'tivesse', 'tivessem', 'tivéssemos', 'tiveste', 'tivestes', 'toda', 'todas', 'todavia', 'todo', 'todos', 'trabalho', 'três', 'treze', 'tu', 'tua', 'tuas', 'tudo', 'última', 'últimas', 'último', 'últimos', 'um', 'uma', 'umas', 'uns', 'vai', 'vais', 'vão', 'vários', 'vem', 'vêm', 'vendo', 'vens', 'ver', 'vez', 'vezes', 'viagem', 'vindo', 'vinte', 'vir', 'você', 'vocês', 'vos', 'vós', 'vossa', 'vossas', 'vosso', 'vossos', 'zero', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '_' ]

In [4]:
class DataFormatter:
    
    def __init__(self):
        self.data = DataExtractor().data
    
    def data_split(self):
        y = self.data['category']
        X_train, X_test, y_train, y_test = train_test_split(self.data['query'], y, test_size=0.3, random_state=123)
        return X_train, X_test, y_train, y_test
        
    def count_vectorizer(self):
        
        count_vectorizer = CountVectorizer(lowercase=True, stop_words=stop_words) 
        count_train = count_vectorizer.fit_transform(self.data_split()[0])
        count_test = count_vectorizer.transform(self.data_split()[1])
        return count_train, count_test, count_vectorizer

# 3. Modeling

In [5]:
class Modeler:
    
    def classifier(self):    
        nb_classifier = MultinomialNB()
        nb_classifier.fit(DataFormatter().count_vectorizer()[0], DataFormatter().data_split()[2])
        return nb_classifier

# 4. Model Validation

In [6]:
class ModelValidator:
    
    def __init__(self):
        self.model = Modeler().classifier()
    
    def prediction(self):
        y_pred = self.model.predict(DataFormatter().count_vectorizer()[1])
        return y_pred
    
    def metrics(self):
        metrics_report = classification_report(DataFormatter().data_split()[3].values, self.prediction())
        f = open(os.getenv('METRICS_PATH'), 'w')
        f.write(f'Test set metrics:\n')
        f.write(f'\n{metrics_report}\n')
        f.close()

# 5. Data engineering and model exportation

In [7]:
class ModelExporter:
    
    def __init__(self):
        self.model = Modeler().classifier()    
        self.data_engineering = DataFormatter().count_vectorizer()[2]
        pickle.dump(self.model, open(os.getenv('MODEL_PATH'),'wb'))
        pickle.dump(self.data_engineering, open(os.getenv('DATA_ENGINEERING_PATH'),'wb'))

In [8]:
# pipeline execution
if __name__ == '__main__':
    ModelValidator().metrics()
    ModelExporter()